In [ ]:
from datetime import datetime, timedelta
import torch
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import numpy as np
from vctr.models.lstm.data import get_train_and_val_loaders, get_train_and_val_loaders_with_close

from vctr.models.lstm.defaults import SEQUENCE_LENGTH
from vctr.utils.label_tracker import LabelTracker
from vctr.trading.coins import tradable_coins
from vctr.utils.pf_stats import get_plot_and_pf, print_stats
from vctr.models.lstm.actions import load_model, dynamic_threshold_adjustment
from vctr.data.lstm_preprocessor import preprocess_data
from vctr.data.data_loader import get_data_with_features_and_labels

import faulthandler

faulthandler.enable()

model = load_model('latest')
# model = load_model('lstm-mk-943-REN-XTZ')
# model = load_model('lstm-mk-820-BTC-highest-profit')

timeframes = ['15m']
start = datetime.now() - timedelta(days=180)
end = datetime.now() - timedelta(days=90)
label_args = (0.035, 0.005)
batch_size = 1

window_size = SEQUENCE_LENGTH
sequence_length = SEQUENCE_LENGTH


class WindowedDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, window_size):
        self.X = torch.tensor(X).float().to('mps')
        self.y = torch.tensor(y).long().to('mps')
        self.window_size = window_size

    def __getitem__(self, index):
        X_slice = self.X[index : index + self.window_size]
        y_slice = self.y[index : index + self.window_size]
        return X_slice, y_slice

    def __len__(self):
        return len(self.X) - self.window_size + 1


returns = []
tracker = LabelTracker('win_rates.json')

for symbol in tradable_coins:
    # df = get_data_with_features_and_labels(
    #     symbol=symbol,
    #     timeframes=timeframes,
    #     label_args=label_args,
    #     start=start,
    #     # end=end,
    #     separate=False,
    # )

    train_loader, val_loader, data = get_train_and_val_loaders(
        start=start,
        end=end,
        symbol=symbol,
        timeframes=timeframes,
        lookback=sequence_length,
        label_args=label_args,
        batch_size=batch_size,
        test_pct=0,
        return_data=True,
    )

    # Make `data` the same length as the training data
    df = data[-len(train_loader.dataset) :]

    # X_cls = df.drop(['label'], axis=1)
    # X_cls = StandardScaler().fit_transform(X_cls)

    # X, y = preprocess_data(df.drop(['label'], axis=1), df['label'], lookback=sequence_length, torch=True)

    # dataset = WindowedDataset(X, y, window_size)
    # loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

    # all_logits = []
    # for inputs, targets in tqdm(loader, colour='green', ncols=65):
    #     with torch.no_grad():
    #         logits = model(inputs.to('mps')[0])
    #         all_logits.append(logits[-1:])
    all_logits = []
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to('mps'), target.to('mps')
            logits = model(data)
            all_logits.append(logits)

    all_logits = torch.cat(all_logits, dim=0)
    all_predictions = torch.argmax(all_logits, dim=1).cpu().numpy()
    all_predictions = dynamic_threshold_adjustment(all_logits, [0.96, 0.02, 0.02])

    df = df[-len(all_predictions) :].copy()
    df['pred'] = pd.Series(all_predictions, index=df.index)

    fig, pf = get_plot_and_pf(df, 'pred', pf_params={'sl_stop': None}, width=1050, height=600)

    fig.data[0].line.width = 1
    fig.data[0].opacity = 0.75
    fig.show()

    print(f'Symbol: {symbol}')
    print_stats(pf)
    returns.append(pf.total_return)
    tracker.track(symbol, pf.sortino_ratio)
    print(f'Average return: {np.mean(returns) * 100:.0f}%')

print(model.name)


In [ ]:
from vctr.models.lstm.actions import save_model


save_model(model, f'{model.name}-saved')